<a href="https://colab.research.google.com/github/tilakparth/Automatic-Image-Captioning/blob/main/Automatic_Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
import string
import glob
import os

In [3]:
def load_document(filename):
	# opening the file as read only
	file = open(filename, 'r')
	# reading all text
	text = file.read()
	# closing the file
	file.close()
	return text

filename = "/content/drive/MyDrive/flicker8k/Flickr_Data/Flickr_TextData/Flickr8k.token.txt"
# loading descriptions
doc = load_document(filename)
print(doc[:300])

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the s


In [5]:
def load_descriptions(doc):
	mapping = dict()
	# processing lines
	for line in doc.split('\n'):
		# spliting line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# taking the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extracting filename from image id
		image_id = image_id.split('.')[0]
		# converting description tokens back to string
		image_desc = ' '.join(image_desc)
		# creating the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# storing description
		mapping[image_id].append(image_desc)
	return mapping

# parsing descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 8092 


In [6]:
list(descriptions.keys())[0:10]

['1000268201_693b08cb0e',
 '1001773457_577c3a7d70',
 '1002674143_1b742ab4b8',
 '1003163366_44323f5815',
 '1007129816_e794419615',
 '1007320043_627395c3d8',
 '1009434119_febe49276a',
 '1012212859_01547e3f17',
 '1015118661_980735411b',
 '1015584366_dfcec3c85a']

In [7]:
descriptions['1007320043_627395c3d8']

['A child playing on a rope net .',
 'A little girl climbing on red roping .',
 'A little girl in pink climbs a rope bridge at the park .',
 'A small child grips onto the red ropes at the playground .',
 'The small child climbs on a red ropes on a playground .']

In [8]:
def clean_descriptions(descriptions):
	# preparing translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenizing
			desc = desc.split()
			# converting to lower case
			desc = [word.lower() for word in desc]
			# removing punctuation from each token
			desc = [w.translate(table) for w in desc]
			# removing hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# removing tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# storing as string
			desc_list[i] =  ' '.join(desc)

# cleaning descriptions
clean_descriptions(descriptions)

In [9]:
descriptions['1007320043_627395c3d8']

['child playing on rope net',
 'little girl climbing on red roping',
 'little girl in pink climbs rope bridge at the park',
 'small child grips onto the red ropes at the playground',
 'the small child climbs on red ropes on playground']

In [10]:
# converting the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# building a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarizing vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 8763


In [12]:
# saving descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, 'descriptions.txt')

In [ ]:
# loading a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_document(filename)
	dataset = list()
	# processing line by line
	for line in doc.split('\n'):
		# skiping empty lines
		if len(line) < 1:
			continue
		# geting the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# loading training dataset (6K)
filename = '/content/drive/MyDrive/flicker8k/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
train = load_set(filename)
print(train)
print('Dataset: %d' % len(train))

In [46]:
# Below path contains all the images
images = '/content/drive/MyDrive/flicker8k/Flickr_Data/Images/'
# Creating a list of all image names in the directory
img = glob.glob(images + '*?.jpg')

In [47]:
# Below file conatains the names of images to be used in train data
train_images_file = '/content/drive/MyDrive/flicker8k/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
# Reading the train image names in a set
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))
# Creating a list of all the training images with their full path names

train_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in train_images: # Checking if the image belongs to training set
        train_img.append(i) # Adding it to the list of train images

In [49]:
# Below file conatains the names of images to be used in test data
test_images_file = '/content/drive/MyDrive/flicker8k/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'
# Read the validation image names in a set# Read the test image names in a set
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images